In [ ]:
import fitz  # PyMuPDF
import os

def pdf_to_images(pdf_path, output_folder="pages"):
    # Tạo thư mục output nếu chưa có
    os.makedirs(output_folder, exist_ok=True)

    # Mở PDF
    doc = fitz.open(pdf_path)
    total_pages = len(doc)  # Lưu số trang trước khi đóng

    for page_num in range(total_pages):
        # Lấy trang
        page = doc.load_page(page_num)

        # Chuyển thành ảnh với độ phân giải cao
        matrix = fitz.Matrix(2, 2)  # Zoom 2x để có chất lượng tốt hơn
        pix = page.get_pixmap(matrix=matrix)

        # Lưu ảnh
        output_path = f"{output_folder}/page_{page_num + 1:03d}.png"
        pix.save(output_path)
        print(f"Đã lưu: {output_path}")

    doc.close()
    print(f"Hoàn thành! Đã tách {total_pages} trang vào thư mục '{output_folder}'")

# Sử dụng
pdf_path = "raw_book.pdf"
pdf_to_images(pdf_path)

In [ ]:
!pip install pytesseract pillow


In [ ]:
import pytesseract
from PIL import Image
import os
import glob

# Nếu vẫn lỗi, thêm dòng này để chỉ định đường dẫn
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def ocr_images_to_text(images_folder="pages", output_file="extracted_text.txt"):
    image_files = sorted(glob.glob(f"{images_folder}/*.png"))
    print(f"Tìm thấy {len(image_files)} ảnh để xử lý...")

    all_text = ""

    for i, image_path in enumerate(image_files):
        print(f"Đang xử lý: {image_path} ({i+1}/{len(image_files)})")

        try:
            img = Image.open(image_path)
            # Sử dụng cả tiếng Việt và tiếng Anh
            text = pytesseract.image_to_string(img, lang='vie+eng')

            all_text += f"\n{'='*50}\n"
            all_text += f"TRANG {i+1} - {os.path.basename(image_path)}\n"
            all_text += f"{'='*50}\n"
            all_text += text
            all_text += "\n"
            if i == 5:
                print('test break')
                break
            print(f"✓ Hoàn thành trang {i+1}")

        except Exception as e:
            print(f"✗ Lỗi trang {i+1}: {e}")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(all_text)

    print(f"\n🎉 Hoàn thành! Text đã lưu vào: {output_file}")
    return all_text

# Chạy OCR
text = ocr_images_to_text("pages", "extracted_text.txt")